In [1]:
import sys, os
sys.path.append('../..')
from tools import ML
from tools.instruments import instruments
from tools.display import ohlc_chart, highchart
import h5py
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import time

In [2]:
%load_ext autoreload
%aimport tools.ML
%aimport tools.display
%autoreload 2

### 0. Parameters

In [5]:
period= 50
filepath = os.path.join('..','..','data','trend index','training set', 'trend_50day.hdf')
training_period = '2009-12-31'

In [20]:
file = h5py.File(filepath, mode='r')
file2 = h5py.File('emp', mode='r')

In [21]:
file2['ohlc'].shape

(422, 4, 50)

In [22]:
file['train'].shape

(422, 50, 4)

In [10]:
ohlc = np.moveaxis(ohlc, 1, 2)

In [12]:
ohlc.shape

(422, 4, 50)

In [18]:
file2.create_dataset('expect', ohlc.shape, data=ohlc)

<HDF5 dataset "expect": shape (422,), type "<i4">

In [19]:
file2.close()

### 1. Gathering Data

In [4]:
quotes = instruments.quotes(fields='ohlc')[:training_period].copy()
symbols = list(quotes.columns.levels[0])

In [66]:
file = h5py.File(filepath, mode='a')

num_dim = 4 #open, high, low, close
if 'train' not in file.keys():
        file.create_dataset('train', (0, window, num_dim), maxshape=(None, window, num_dim), dtype='float32', compression='gzip')
        file.create_dataset('trend', (0,), maxshape=(None,), dtype='i')
        file.create_dataset('guess', (0,), maxshape=(None,), dtype='i')
        
Train, Trend, Guess  = file['train'], file['trend'], file['guess']


while True:
    # 상품 랜덤 선택
    symbol = np.random.choice(symbols)
    name = instruments[symbol].name
    quote = quotes[symbol].dropna()
    #print(instruments[symbol].name)
    
    # 10세트 데이터 랜덤 선택. quote 없으면 넘어감
    if len(quote) > window+100:
        indexes = np.random.randint(0, len(quote)-window, 10)
    else:
        continue
        
    for index in indexes:
        # 데이터 셋 선택 및 규격확
        data = quote.iloc[index:index+window]
        train_data = (data - data.mean())/data.std().mean()
        highchart([{ 'data':data, 'name':name, 'type':'candlestick'}], height=600)
        
        time.sleep(0.1)
        trend = input("추세? 하락(-1), 없음(0), 상승(1): ")
        guess = input("예측? 하락(-1), 관망(0), 상승(1): ")
        if (trend not in ['-1','0','1']) or (guess not in ['-1','0','1']):
            print(f"You pressed trend:{trend} and guess: {guess}")
            break
        else:
            size = Train.shape[0]
            Train.resize(size+1, axis=0)
            Trend.resize(size+1, axis=0)
            Guess.resize(size+1, axis=0)

            Train[size] = data.values
            Trend[size] = int(trend)
            Guess[size] = int(guess)

        clear_output(wait=True)
    
    if (trend not in ['-1','0','1']) or (guess not in ['-1','0','1']):
            break
file.close()

추세? 하락(-1), 없음(0), 상승(1): x
예측? 하락(-1), 관망(0), 상승(1): x
You pressed trend:x and guess: x


### 2. Verification

In [67]:
from collections import Counter

In [69]:
file = h5py.File(filepath, mode='r')

In [70]:
print(file['train'].shape)
print(file['trend'].shape)
print(file['guess'].shape)

(422, 50, 4)
(422,)
(422,)


In [71]:
Y = file['trend']
Counter(Y)

Counter({0: 178, -1: 103, 1: 141})

In [72]:
file.close()